# Makine Öğrenimi ile Fiyat Konut Tahmini: Makine Öğrenme Modelinin Kurulması

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
df = pd.read_excel(r"C:\Users\Monster\Desktop\konut-fiyat-tahmin-projesi\Veri Seti\istanbul_konut2.xlsx")

In [3]:
# Eksik değerlerin toplamını ve oranını göster
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100

# Eksik değer tablosu oluştur
missing_data = pd.DataFrame({'Eksik Değerler': missing_values, 'Eksik Oranı (%)': missing_percent})
print(missing_data)

               Eksik Değerler  Eksik Oranı (%)
Fiyat                       0         0.000000
İlce                        0         0.000000
Mahalle                     0         0.000000
Metre Kare                  0         0.000000
Oda Sayısı                  2         0.006988
Yaş                        24         0.083852
Bulunduğu Kat             936         3.270212


In [4]:
# 'Yaş' sütunundaki eksik değerleri medyan ile doldur
df['Yaş'] = df['Yaş'].fillna(df['Yaş'].median())
# 'Bulunduğu Kat' sütunundaki eksik değerleri mod ile doldur
df['Bulunduğu Kat'] = df['Bulunduğu Kat'].fillna(df['Bulunduğu Kat'].mode()[0])
# Oda Sayısı sütunundaki eksik değerleri mod (en sık görülen değer) ile doldur
most_frequent_value = df['Oda Sayısı'].mode()[0]
df['Oda Sayısı'] = df['Oda Sayısı'].fillna(most_frequent_value)
print(df.isnull().sum())

Fiyat            0
İlce             0
Mahalle          0
Metre Kare       0
Oda Sayısı       0
Yaş              0
Bulunduğu Kat    0
dtype: int64


In [5]:
df

,Fiyat,İlce,Mahalle,Metre Kare,Oda Sayısı,Yaş,Bulunduğu Kat
0,5000000,Adalar,Burgazada Mah,55,2.0,24.0,0.0
1,7500000,Adalar,Nizam Mah,65,2.0,31.0,0.0
2,4750000,Adalar,Nizam Mah,60,2.0,34.0,0.0
3,7000000,Adalar,MadeMah,75,2.0,35.0,0.0
4,4200000,Adalar,Nizam Mah,51,2.0,38.0,0.0
...,...,...,...,...,...,...,...
28617,4150000,Zeytinburnu,VeliefendMah.,150,6.0,29.0,1.0
28618,4500000,Zeytinburnu,VeliefendMah.,170,6.0,30.0,1.0
28619,17000000,Zeytinburnu,Gökalp Mah.,350,12.0,16.0,0.0
28620,19000000,Zeytinburnu,Beştelsiz Mah.,500,12.0,28.0,0.0


# Target Encoding (Mean Encoding)

Her bir ilçe/mahalle, o ilçe/mahalledeki ortalama konut fiyatı ile değiştirilir

In [6]:
# Target Encoding uygulama
def target_encode(df, target_col):
    # İlce için ortalama fiyatları hesapla
    ilce_means = df.groupby('İlce')[target_col].mean()
    df['Ilce_encoded'] = df['İlce'].map(ilce_means)

    # Mahalle için ortalama fiyatları hesapla
    mahalle_means = df.groupby('Mahalle')[target_col].mean()
    df['Mahalle_encoded'] = df['Mahalle'].map(mahalle_means)

    return df

# Target Encoding'i uygula
df_encoded = target_encode(df.copy(), 'Fiyat')

# Özellikler (X) ve hedef değişkeni (y) belirleyelim
X = df_encoded[['Metre Kare', 'Oda Sayısı', 'Yaş', 'Bulunduğu Kat', 'Ilce_encoded', 'Mahalle_encoded']]
y = df_encoded['Fiyat']

# Veriyi eğitim ve test olarak ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest modelini oluştur ve eğit
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Test verisi üzerinde tahmin yap
y_pred = model.predict(X_test)

# R² skorunu hesapla
r2 = r2_score(y_test, y_pred)

# Sonucu yazdır
print("Target Encoding uygulanmış veri seti ile R² değeri (Random Forest):")
print("R² Score:", r2)


Target Encoding uygulanmış veri seti ile R² değeri (Random Forest):
R² Score: 0.8296936501480221


# Outlier Değerleri Temizleme

# Hibrid Yöntem - (IQR + Manuel Veri Temizleme)

In [7]:
df_new=df_encoded.copy()

In [8]:
df_new=df_encoded[df_encoded["Oda Sayısı"]<8]
df_new=df_encoded[df_encoded["Yaş"]<57]
df_new = df_encoded[(df_encoded["Metre Kare"] < 270) & (df_encoded["Metre Kare"] > 40)]

In [9]:
X_clean = df_new[['Metre Kare', 'Oda Sayısı', 'Yaş', 'Bulunduğu Kat', 'Ilce_encoded', 'Mahalle_encoded']]
y_clean = df_new['Fiyat']

# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X_clean, y_clean, test_size=0.2, random_state=42)

# Modeli eğitme
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Modelin performansını değerlendirme
y_pred = rf_model.predict(X_test)
r2 = r2_score(y_test, y_pred)

print(f'Manuel Temizleme Sonrası R^2 Skoru: {r2:.4f}')
print(f'Manuel Temizleme Sonrası Veri sayısı: {len(X_clean)}')

Manuel Temizleme Sonrası R^2 Skoru: 0.8334
Manuel Temizleme Sonrası Veri sayısı: 27961


In [10]:
df_clean = pd.concat([X_clean, y_clean], axis=1)
# İlçe bazında Fiyat, Metre Kare, Oda Sayısı ve Yaş değişkenleri için ortalama ve standart sapma hesaplama
stats = df_clean.groupby('Ilce_encoded').agg(
    Fiyat_mean=('Fiyat', 'mean'), Fiyat_std=('Fiyat', 'std'),
    MetreKare_mean=('Metre Kare', 'mean'), MetreKare_std=('Metre Kare', 'std'),
    OdaSayisi_mean=('Oda Sayısı', 'mean'), OdaSayisi_std=('Oda Sayısı', 'std'),
    Yas_mean=('Yaş', 'mean'), Yas_std=('Yaş', 'std')
).reset_index()

# Aykırı değer eşiğini hesaplama ve kontrol etme
def is_outlier(row, stats):
    ilce_stats = stats[stats['Ilce_encoded'] == row['Ilce_encoded']].iloc[0]
    
    # Eşik değerler
    fiyat_threshold = ilce_stats['Fiyat_mean'] + 2 * ilce_stats['Fiyat_std'] + 1
    metrekare_threshold = ilce_stats['MetreKare_mean'] + 2 * ilce_stats['MetreKare_std'] + 1
    odasayisi_threshold = ilce_stats['OdaSayisi_mean'] + 2 * ilce_stats['OdaSayisi_std'] + 1
    yas_threshold = ilce_stats['Yas_mean'] + 2 * ilce_stats['Yas_std'] + 1

    # Aykırı değer kontrolü
    if (row['Fiyat'] > fiyat_threshold or 
        row['Metre Kare'] > metrekare_threshold or 
        row['Oda Sayısı'] > odasayisi_threshold or 
        row['Yaş'] > yas_threshold):
        return True
    return False

# Aykırı değerleri filtreleme
filtered_data = df_clean[~df_clean.apply(lambda row: is_outlier(row, stats), axis=1)]

# Sonuç
print(f"Orijinal veri boyutu: {len(df_clean)}")
print(f"İlçe Bazlı Temizlenme sonucu  veri boyutu: {len(filtered_data)}")

Orijinal veri boyutu: 27961
İlçe Bazlı Temizlenme sonucu  veri boyutu: 24973


In [11]:
filtered_data

,Metre Kare,Oda Sayısı,Yaş,Bulunduğu Kat,Ilce_encoded,Mahalle_encoded,Fiyat
0,55,2.0,24.0,0.0,1.047431e+07,1.064654e+07,5000000
1,65,2.0,31.0,0.0,1.047431e+07,1.104514e+07,7500000
2,60,2.0,34.0,0.0,1.047431e+07,1.104514e+07,4750000
3,75,2.0,35.0,0.0,1.047431e+07,9.944185e+06,7000000
4,51,2.0,38.0,0.0,1.047431e+07,1.104514e+07,4200000
...,...,...,...,...,...,...,...
28604,150,5.0,30.0,0.0,4.050061e+06,1.965306e+06,3000000
28607,125,5.0,31.0,0.0,4.050061e+06,4.250833e+06,2650000
28608,150,5.0,31.0,1.0,4.050061e+06,3.729308e+06,4449000
28614,150,6.0,25.0,1.0,4.050061e+06,3.708659e+06,4200000


In [12]:
# df_clean = pd.concat([X_clean, y_clean], axis=1)

In [13]:
# Özellikler ve hedef değişkeni belirleme
X = filtered_data[['Metre Kare', 'Oda Sayısı', 'Yaş', 'Bulunduğu Kat', 'Ilce_encoded', 'Mahalle_encoded']]
y = filtered_data['Fiyat']

# IQR yöntemiyle aykırı değerleri temizleme
Q1 = X.quantile(0.25)
Q3 = X.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 2 * IQR
upper_bound = Q3 + 2 * IQR
mask = ~((X < lower_bound) | (X > upper_bound)).any(axis=1)
X_clean = X[mask]
y_clean = y[mask]

# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X_clean, y_clean, test_size=0.2, random_state=42)

# Modeli eğitme
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Modelin performansını değerlendirme
y_pred = rf_model.predict(X_test)
r2 = r2_score(y_test, y_pred)

print(f'R^2 Skoru: {r2:.4f}')
print(f'IQR sonrası Veri boyutu: {len(X_clean)}')

R^2 Skoru: 0.8932
IQR sonrası Veri boyutu: 24129


In [14]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# df_clean = pd.concat([X_clean, y_clean], axis=1)
# # DataFrame yüklü olduğunu varsayıyoruz: 'df'
# def veri_kesif_analizi(df_clean):
    
#     # İlçe Bazında Fiyat Dağılımı
#     plt.figure(figsize=(15, 6))
#     df_clean.boxplot(column='Fiyat', by='Ilce_encoded', rot=90)
#     plt.title('İlçe Bazında Fiyat Dağılımı')
#     plt.suptitle('')  # Otomatik başlığı kaldırır
#     plt.ylabel('Fiyat')
#     plt.tight_layout()
#     plt.show()
    
#     plt.figure(figsize=(12, 6))
    
#     # 1. Grafik: Mülk Büyüklüğü Dağılımı
#     plt.subplot(1, 2, 1)
#     df_clean['Metre Kare'].hist(bins=30)
#     plt.title('Mülk Büyüklüğü Dağılımı')
#     plt.xlabel('Metrekare')
    
#     # 2. Grafik: Mülk Büyüklüğü Kutu Grafiği
#     plt.subplot(1, 2, 2)
#     sns.boxplot(x=df_clean['Metre Kare'])
#     plt.title('Mülk Büyüklüğü Kutu Grafiği')
    
#     plt.tight_layout()
#     plt.show()
    
#     # Yeni bir figure açarak ikinci grup grafikleri yerleştiriyoruz
#     plt.figure(figsize=(12, 6))
    
#     # 3. Grafik: Oda Sayısı Kutu Grafiği
#     plt.subplot(1, 2, 1)
#     sns.boxplot(x=df_clean['Oda Sayısı'])
#     plt.title('Oda Sayısı Kutu Grafiği')
    
#     # 4. Grafik: Yaş Kutu Grafiği
#     plt.subplot(1, 2, 2)
#     sns.boxplot(x=df_clean['Yaş'])
#     plt.title('Bina Yaşı Kutu Grafiği')
    
#     plt.tight_layout()
#     plt.show()
    
#     # Fiyat ve Metrekare İlişkisi
#     plt.figure(figsize=(10, 6))
#     plt.scatter(df_clean['Metre Kare'], df_clean['Fiyat'], alpha=0.5)
#     plt.title('Fiyat ve Mülk Büyüklüğü İlişkisi')
#     plt.xlabel('Metrekare')
#     plt.ylabel('Fiyat')
#     plt.tight_layout()
#     plt.show()
    

# # Fonksiyonu Çağır
# veri_kesif_analizi(df_clean)

# Makine Öğrenmesi Modelelerinin Kurulması

## 1. Temel Regresyon Modelleri

📌 Basit ve hızlı modellerdir, ancak karmaşık ilişkileri yakalamada sınırlıdır.

#### ✅ Linear Regression (Doğrusal Regresyon) – Veriler arasındaki doğrusal ilişkileri modellemek için temel modeldir.
#### ✅ Ridge Regression – Aşırı öğrenmeyi önlemek için L2 düzenlileştirme uygular.
#### ✅ Lasso Regression – Özellik seçimi yaparak bazı değişkenleri sıfır yapan L1 düzenlileştirme uygular.
#### ✅ ElasticNet Regression – Hem L1 hem de L2 düzenlileştirme kullanarak esneklik sunar.

In [16]:
df_clean = pd.concat([X_clean, y_clean], axis=1)

In [17]:
df_clean

,Metre Kare,Oda Sayısı,Yaş,Bulunduğu Kat,Ilce_encoded,Mahalle_encoded,Fiyat
0,55,2.0,24.0,0.0,1.047431e+07,1.064654e+07,5000000
1,65,2.0,31.0,0.0,1.047431e+07,1.104514e+07,7500000
2,60,2.0,34.0,0.0,1.047431e+07,1.104514e+07,4750000
3,75,2.0,35.0,0.0,1.047431e+07,9.944185e+06,7000000
4,51,2.0,38.0,0.0,1.047431e+07,1.104514e+07,4200000
...,...,...,...,...,...,...,...
28604,150,5.0,30.0,0.0,4.050061e+06,1.965306e+06,3000000
28607,125,5.0,31.0,0.0,4.050061e+06,4.250833e+06,2650000
28608,150,5.0,31.0,1.0,4.050061e+06,3.729308e+06,4449000
28614,150,6.0,25.0,1.0,4.050061e+06,3.708659e+06,4200000


In [18]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
df_clean = pd.concat([X_clean, y_clean], axis=1)
X_train = df_clean[['Metre Kare', 'Oda Sayısı', 'Yaş', 'Bulunduğu Kat', 'Ilce_encoded', 'Mahalle_encoded']]
y_train = df_clean['Fiyat']
# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X_clean, y_clean, test_size=0.2, random_state=42)



# Mean Absolute Percentage Error (MAPE) hesaplayan fonksiyon
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Modelleri tanımlama
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),  # L2 düzenlileştirme parametresi
    "Lasso Regression": Lasso(alpha=0.1),  # L1 düzenlileştirme parametresi
    "ElasticNet Regression": ElasticNet(alpha=0.1, l1_ratio=0.5)  # L1 ve L2 kombinasyonu
}

# Her model için eğitim ve değerlendirme
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)  # Modeli eğitme
    y_pred = model.predict(X_test)  # Test seti üzerinde tahmin
    
    # Performans metrikleri
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)  # MAPE hesaplama
    
    results[name] = {"MSE": mse, "R² Score": r2, "MAPE (%)": mape}

# Sonuçları DataFrame olarak gösterme
results_df = pd.DataFrame(results).T
print(results_df)

                                MSE  R² Score   MAPE (%)
Linear Regression      4.218131e+12  0.768247  38.618741
Ridge Regression       4.218129e+12  0.768247  38.618765
Lasso Regression       4.218131e+12  0.768247  38.618742
ElasticNet Regression  4.216822e+12  0.768319  38.639161


In [19]:
X_train

,Metre Kare,Oda Sayısı,Yaş,Bulunduğu Kat,Ilce_encoded,Mahalle_encoded
24168,98,3.0,6.0,0.0,5.327358e+06,8.861630e+06
14870,100,3.0,0.0,6.0,1.206458e+07,1.353250e+07
23243,176,4.0,5.0,0.0,4.495740e+06,4.836000e+06
16636,150,4.0,35.0,8.0,1.206458e+07,7.880303e+06
10312,100,3.0,3.0,3.0,2.759087e+06,2.895897e+06
...,...,...,...,...,...,...
25710,70,2.0,30.0,0.0,6.467187e+06,5.375862e+06
6160,110,3.0,0.0,4.0,4.709827e+06,5.546995e+06
1011,180,5.0,11.0,0.0,6.383488e+06,4.042614e+06
19048,65,2.0,10.0,0.0,2.596074e+06,4.296346e+06


## 2. Ağaç Tabanlı Modeller

#### ✅ Basic Random Forest
#### ✅ Extra Trees
#### ✅ Gradient Boosting
#### ✅ XGBoost
#### ✅ LightGBM
#### ✅ CatBoost
#### ✅ AdaBoost
#### ✅ Bagging
#### ✅ Deep Gradient Boosting 

In [23]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # 0'a bölünmeyi engellemek için küçük bir değer ekliyoruz
    epsilon = 1e-10
    return np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

def train_and_evaluate_models(X_train, X_test, y_train, y_test):
    results = {}
    
    # 1. Basit Random Forest
    rf1 = RandomForestRegressor(n_estimators=100, random_state=42)
    rf1.fit(X_train, y_train)
    results['Basic Random Forest'] = rf1
    
    
    # 3. Extra Trees
    et = ExtraTreesRegressor(
        n_estimators=100,
        random_state=42
    )
    et.fit(X_train, y_train)
    results['Extra Trees'] = et
    
    # 4. Gradient Boosting
    gb = GradientBoostingRegressor(
        n_estimators=100,
        learning_rate=0.1,
        random_state=42
    )
    gb.fit(X_train, y_train)
    results['Gradient Boosting'] = gb
    
    # 5. XGBoost
    xgb = XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        random_state=42
    )
    xgb.fit(X_train, y_train)
    results['XGBoost'] = xgb
    
    # 6. LightGBM
    lgbm = LGBMRegressor(
        n_estimators=100,
        learning_rate=0.1,
        random_state=42
    )
    lgbm.fit(X_train, y_train)
    results['LightGBM'] = lgbm
    
    # 7. CatBoost
    cb = CatBoostRegressor(
        n_estimators=100,
        learning_rate=0.1,
        random_state=42,
        verbose=False
    )
    cb.fit(X_train, y_train)
    results['CatBoost'] = cb
    
    # 8. AdaBoost
    ada = AdaBoostRegressor(
        DecisionTreeRegressor(max_depth=4),
        n_estimators=100,
        random_state=42
    )
    ada.fit(X_train, y_train)
    results['AdaBoost'] = ada
    
    # 9. Bagging with Decision Trees
    bag = BaggingRegressor(
        DecisionTreeRegressor(max_depth=6),
        n_estimators=100,
        random_state=42
    )
    bag.fit(X_train, y_train)
    results['Bagging'] = bag
    
    # 10. Gradient Boosting with Deep Trees
    gb_deep = GradientBoostingRegressor(
        n_estimators=100,
        max_depth=8,
        learning_rate=0.05,
        subsample=0.8,
        random_state=42
    )
    gb_deep.fit(X_train, y_train)
    results['Deep Gradient Boosting'] = gb_deep
    
    # Modelleri değerlendir
    print("Model Performansları:")
    print("-" * 60)
    print(f"{'Model':30} {'MSE':>10} {'R2':>10} {'MAPE':>10}")
    print("-" * 60)
    
    for name, model in results.items():
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)
        
        print(f"{name:30} {mse:10.4f} {r2:10.4f} {mape:10.2f}%")
    
    print("-" * 60)
    return results

# Kullanım örneği:
results = train_and_evaluate_models(X_train, X_test, y_train, y_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001421 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 528
[LightGBM] [Info] Number of data points in the train set: 19303, number of used features: 6
[LightGBM] [Info] Start training from score 5578966.878724
Model Performansları:
------------------------------------------------------------
Model                                 MSE         R2       MAPE
------------------------------------------------------------
Basic Random Forest            1943942434180.6855     0.8932      17.07%
Extra Trees                    2004679494114.4153     0.8899      17.02%
Gradient Boosting              2789631586827.8052     0.8467      26.00%
XGBoost                        2081732820992.0000     0.8856      20.24%
LightGBM  

## 3. Yapay Sinir Ağları (Deep Learning)

#### ✅ Multi-Layer Perceptron (MLP) Regressor – Sinir ağlarıyla tahminleme yapan bir modeldir.

In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor

def mean_absolute_percentage_error(y_true, y_pred):

    epsilon = 1e-10  # Sıfıra bölünmeyi engellemek için
    return np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

def train_basic_mlp(X_train, X_test, y_train, y_test, 
                    hidden_layers=(100,),  # Gizli katman yapısı
                    activation='relu',      # Aktivasyon fonksiyonu
                    max_iter=200,          # Maksimum iterasyon sayısı
                    learning_rate=0.001,   # Öğrenme oranı
                    random_state=42):      # Rastgelelik için seed

    # Veriyi ölçeklendir
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Modeli oluştur
    model = MLPRegressor(
        hidden_layer_sizes=hidden_layers,
        activation=activation,
        solver='adam',
        alpha=0.0001,  # L2 regularizasyon parametresi
        batch_size='auto',
        learning_rate='constant',
        learning_rate_init=learning_rate,
        max_iter=max_iter,
        shuffle=True,
        random_state=random_state,
        early_stopping=True,  # Erken durma
        validation_fraction=0.1,  # Validation seti oranı
        n_iter_no_change=10,  # Kaç iterasyon sonra early stopping kontrolü
        verbose=False
    )
    
    # Modeli eğit
    model.fit(X_train_scaled, y_train)
    
    # Tahmin yap
    y_pred_train = model.predict(X_train_scaled)
    y_pred_test = model.predict(X_test_scaled)
    
    # Metrikleri hesapla
    metrics = {
        'test_mse': mean_squared_error(y_test, y_pred_test),
        'test_r2': r2_score(y_test, y_pred_test),
        'test_mape': mean_absolute_percentage_error(y_test, y_pred_test),
        'n_iterations': model.n_iter_,
        'loss_curve': model.loss_curve_
    }
    
    # Sonuçları yazdır
    print("\nModel Performansı:")
    print("-" * 50)
    print(f"Test MSE      : {metrics['test_mse']:.4f}")
    print(f"Test R2       : {metrics['test_r2']:.4f}")
    print(f"Test MAPE     : {metrics['test_mape']:.2f}%")
    print("-" * 50)
    
    return model, metrics, scaler

# Kullanım örneği:

# Veri setinizi hazırlayın (X_train, X_test, y_train, y_test)

# Modeli eğit
model, metrics, scaler = train_basic_mlp(
    X_train, 
    X_test, 
    y_train, 
    y_test,
    hidden_layers=(100, 50),  # İki gizli katman: 100 ve 50 nöron
    activation='relu',
    max_iter=300,
    learning_rate=0.001
)


Model Performansı:
--------------------------------------------------
Test MSE      : 3814277244940.1655
Test R2       : 0.7904
Test MAPE     : 33.21%
--------------------------------------------------


## 4- Destek Vektör Makineleri (SVM):

#### ✅ Linear SVM
#### ✅ Kernel SVM (RBF, Polynomial)
#### ✅ Stochastic Gradient Descent (SGD) SVM 

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import SGDRegressor

def mean_absolute_percentage_error(y_true, y_pred):
    """MAPE metriği hesaplama"""
    epsilon = 1e-10
    return np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

def train_svm_models(X_train, X_test, y_train, y_test):
    
    # Veriyi ölçeklendir
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Modelleri tanımla
    models = {
        'Linear SVM': LinearSVR(
            epsilon=0.1,
            tol=1e-4,
            C=1.0,
            random_state=42,
            max_iter=1000
        ),
        
        'RBF Kernel SVM': SVR(
            kernel='rbf',
            C=1.0,
            epsilon=0.1,
            gamma='scale'
        ),
        
        'Polynomial Kernel SVM': SVR(
            kernel='poly',
            degree=3,
            C=1.0,
            epsilon=0.1,
            gamma='scale'
        ),
        
        'SGD SVM': SGDRegressor(
            loss='epsilon_insensitive',
            penalty='l2',
            epsilon=0.1,
            max_iter=1000,
            tol=1e-3,
            random_state=42
        )
    }
    
    # Sonuçları saklamak için dictionary
    metrics = {}
    
    # Her modeli eğit ve değerlendir
    print("\nModel Performansları:")
    print("-" * 70)
    print(f"{'Model':20} {'Train MSE':>12} {'Test MSE':>12} {'Train R2':>10} {'Test R2':>10} {'MAPE':>10}")
    print("-" * 70)
    
    for name, model in models.items():
        # Modeli eğit
        model.fit(X_train_scaled, y_train)
        
        # Tahminler
        y_pred_train = model.predict(X_train_scaled)
        y_pred_test = model.predict(X_test_scaled)
        
        # Metrikleri hesapla
        metrics[name] = {
            'train_mse': mean_squared_error(y_train, y_pred_train),
            'test_mse': mean_squared_error(y_test, y_pred_test),
            'train_r2': r2_score(y_train, y_pred_train),
            'test_r2': r2_score(y_test, y_pred_test),
            'test_mape': mean_absolute_percentage_error(y_test, y_pred_test)
        }
        
        # Sonuçları yazdır
        print(f"{name:20} {metrics[name]['train_mse']:12.2f} "
              f"{metrics[name]['test_mse']:12.2f} "
              f"{metrics[name]['train_r2']:10.4f} "
              f"{metrics[name]['test_r2']:10.4f} "
              f"{metrics[name]['test_mape']:10.2f}%")
    
    print("-" * 70)
    return models, metrics, scaler

def predict_with_model(model, scaler, X_new, model_name="Seçilen Model"):

    # Veriyi ölçeklendir
    X_new_scaled = scaler.transform(X_new)
    
    # Tahmin yap
    predictions = model.predict(X_new_scaled)
    
    print(f"\n{model_name} Tahminleri:")
    print("-" * 30)
    for i, pred in enumerate(predictions):
        print(f"Tahmin {i+1}: {pred:.2f}")
    
    return predictions

# Kullanım örneği:

# Veri setinizi hazırlayın (X_train, X_test, y_train, y_test)

# Modelleri eğit
models, metrics, scaler = train_svm_models(X_train, X_test, y_train, y_test)

# En iyi modeli seç (örneğin R2 skoruna göre)
best_model_name = max(metrics.keys(), key=lambda k: metrics[k]['test_r2'])
best_model = models[best_model_name]


Model Performansları:
----------------------------------------------------------------------
Model                   Train MSE     Test MSE   Train R2    Test R2       MAPE
----------------------------------------------------------------------
Linear SVM           48633600330024.05 49807686551035.95    -1.7440    -1.7365      99.41%
